<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Импорт-библиотек" data-toc-modified-id="Импорт-библиотек-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Импорт библиотек</a></span></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Подготовка-данных-к-обучению" data-toc-modified-id="Подготовка-данных-к-обучению-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Подготовка данных к обучению</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Итоговый-вывод" data-toc-modified-id="Итоговый-вывод-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Итоговый вывод</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Импорт библиотек

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle

## Подготовка данных

In [2]:
data = pd.read_csv('/Users/daniilsobolev/Desktop/ml projects/Обучение с учителем/Churn.csv')
data.sample(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
8446,8447,15630511,Picot,691,France,Female,33,6.0,0.00,2,1,0,164074.89,0
9871,9872,15736371,Kennedy,633,France,Female,34,3.0,123034.43,2,1,1,38315.04,0
2016,2017,15633612,Yuryeva,696,France,Male,28,4.0,172646.82,1,1,1,116471.43,0
1490,1491,15761976,Su,797,Spain,Female,31,8.0,0.00,2,1,0,117916.63,0
6705,6706,15782875,Cayley,663,France,Male,33,5.0,157274.36,2,1,1,28531.81,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


В столбце **Tenure** есть пропуски, с чем они могли бы быть связаны - не понятно, заменим пропуски медианным значением.

In [4]:
data['Tenure'] = data['Tenure'].fillna(data['Tenure'].median())

## Подготовка данных к обучению

Для обучения столбцы **RowNumber**, **CustomerId**, **Surname** нам не понядобяться, удлим их, чтобы они не исказили результаты.

**P.S.** Стоило ли так делать? Или же их всё-таки надо оставить? А модель при обучении будет опираться тогда и на них, что в этом случае делать?

In [5]:
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

Далее нам необходимо преобразовать категориальные признаки: **Geography**, **Gender** в численные, воспользуемся техникой прямого кодирования OHE.

In [6]:
data = pd.get_dummies(data, drop_first=True)

Выделим из данных признаки и целевой признак

In [7]:
features = data.drop(['Exited'], axis=1)
target = data['Exited']

Разделим данные на обучающую, валидационную и тестовую выборки

In [8]:
features_train, features_valid, target_train, target_valid = train_test_split(features, target, 
                                                                              test_size=0.40, 
                                                                              random_state=12345)

features_valid, features_test, target_valid, target_test = train_test_split(features_valid, target_valid, 
                                                                              test_size=0.50, 
                                                                              random_state=12345)

In [9]:
print(features_train.shape)
print(features_valid.shape)
print(features_test.shape)
print(target_train.shape)
print(target_valid.shape)
print(target_test.shape)


(6000, 11)
(2000, 11)
(2000, 11)
(6000,)
(2000,)
(2000,)


В данных присутствуют количественные признаки с разным разбросом значений. Поэтому следует масштабировать данные. Проведем стандартизацию данных.

In [10]:
numeric = ['CreditScore', 'Age', 'Balance', 'EstimatedSalary', 'NumOfProducts', 'Tenure']

scaler = StandardScaler()
scaler.fit(features_train[numeric])

features_train[numeric] = pd.DataFrame(scaler.transform(features_train[numeric]), 
                                       features_train[numeric].index, 
                                       features_train[numeric].columns)

features_valid[numeric] = pd.DataFrame(scaler.transform(features_valid[numeric]), 
                                       features_valid[numeric].index, 
                                       features_valid[numeric].columns)

features_test[numeric] = pd.DataFrame(scaler.transform(features_test[numeric]), 
                                       features_test[numeric].index, 
                                       features_test[numeric].columns)



/Users/daniilsobolev/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


Посмотрим на баланс классов:

In [11]:
print(data['Exited'].sum())
print(data['Exited'].count())

2037
10000


То есть класс 0 встречается примерно в 4 раза чаще, чем класс 1. То есть классы не сбалансированны.

Исследуем задачу в рамках несбалансированных классов, а после сбалансируем и проведем повторное исследование.

## Исследование задачи

In [12]:
best_f1_score = 0
best_depth = 0
for depth in range(1, 10):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    
    predictions = model.predict(features_valid)
    probabilities_one_valid = model.predict_proba(features_valid)[:, 1]
    
    if f1_score(target_valid, predictions) > best_f1_score:
        best_f1_score = f1_score(target_valid, predictions)
        roc_auc = roc_auc_score(target_valid, probabilities_one_valid)
        best_depth = depth

print('depth', best_depth)
print('best_f1:', best_f1_score)
print('roc_auc:', roc_auc)

depth 6
best_f1: 0.5696969696969697
roc_auc: 0.8164631712023421


In [13]:
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_train, target_train)

probabilities = model.predict_proba(features_valid)
probabilities_one_valid = model.predict_proba(features_valid)[:, 1]

best_f1_score = 0
best_threshold = 0
for threshold in np.arange(0, 1, 0.02):
    predicted_valid = probabilities_one_valid > threshold 
    
    if f1_score(target_valid, predicted_valid) > best_f1_score: 
        best_f1_score = f1_score(target_valid, predicted_valid)
        roc_auc = roc_auc_score(target_valid, probabilities_one_valid)
        best_threshold = threshold
        
print('threshold', best_threshold)
print('best_f1:', best_f1_score)
print('roc_auc:', roc_auc)

threshold 0.26
best_f1: 0.4989816700610997
roc_auc: 0.7587512627102753


In [14]:
best_f1_score = 0
best_trees = 0
best_depth = 0
for trees in range(1,100):
    for depth in range(1,10):
        model = RandomForestClassifier(random_state=12345, 
                                       n_estimators=trees, 
                                       max_depth=depth)
        model.fit(features_train, target_train)
    
        predictions = model.predict(features_valid)
        probabilities_one_valid = model.predict_proba(features_valid)[:, 1]
        
        if f1_score(target_valid, predictions) > best_f1_score:
            best_f1_score = f1_score(target_valid, predictions)
            roc_auc = roc_auc_score(target_valid, probabilities_one_valid)
            best_trees = trees
            best_depth = depth
        
print('depth', best_depth, 'trees', best_trees)
print('best_f1:', best_f1_score)
print('roc_auc:', roc_auc)

depth 7 trees 5
best_f1: 0.5908419497784343
roc_auc: 0.8381787634815114


Лучшая модель - случайный лес с парметрами n_estimators=5, max_depth=7<br>
Результаты:<br>

best_f1 = 0.5908419497784343<br>
roc_auc = 0.8381787634815114

Это уже дотягивает до удовлетворительного результата, но всё же сбалансируем классы. И сделаем это тремя способами.

## Борьба с дисбалансом

Исходя из прошлого пункта, слеудет, что лучшая модель - случайный лес, будем рассмтривать только её.

In [15]:
best_f1_score = 0
best_trees = 0
best_depth = 0
for trees in range(1,100):
    for depth in range(1,10):
        model = RandomForestClassifier(random_state=12345, 
                                       n_estimators=trees, 
                                       max_depth=depth, 
                                       class_weight='balanced')
        model.fit(features_train, target_train)
    
        predictions = model.predict(features_valid)
        probabilities_one_valid = model.predict_proba(features_valid)[:, 1]
        
        if f1_score(target_valid, predictions) > best_f1_score:
            best_f1_score = f1_score(target_valid, predictions)
            roc_auc = roc_auc_score(target_valid, probabilities_one_valid)
            best_trees = trees
            best_depth = depth
        
print('depth', best_depth, 'trees', best_trees)
print('best_f1:', best_f1_score)
print('roc_auc:', roc_auc)

depth 9 trees 88
best_f1: 0.631578947368421
roc_auc: 0.8550786660940364


При добалансировке классов с помощью аргумента **class_weight** метрики увеличились.

best_f1: 0.631578947368421 <br>
roc_auc: 0.0.8550786660940364

In [16]:
def upsample(features, target, repeat):
    features_zeros = features[target==0]
    features_ones = features[target==1]
    target_zeros = target[target==0]
    target_ones = target[target==1]
    
    features_upsampled = pd.concat([features_zeros] + [features_ones]*repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones]*repeat)
    
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

In [17]:
best_f1_score = 0
best_trees = 0
best_depth = 0
features_upsample, target_upsample = upsample(features_train, target_train, 4)
for trees in range(1,100):
    for depth in range(1,10):
        model = RandomForestClassifier(random_state=12345, 
                                        n_estimators=trees, 
                                        max_depth=depth)
        model.fit(features_upsample, target_upsample)
    
        predictions = model.predict(features_valid)
        probabilities_one_valid = model.predict_proba(features_valid)[:, 1]
        
        if f1_score(target_valid, predictions) > best_f1_score:
            best_f1_score = f1_score(target_valid, predictions)
            roc_auc = roc_auc_score(target_valid, probabilities_one_valid)
            best_trees = trees
            best_depth = depth
        
print('depth', best_depth, 'trees', best_trees)
print('best_f1:', best_f1_score)
print('roc_auc:', roc_auc)

depth 9 trees 67
best_f1: 0.6312433581296492
roc_auc: 0.8527619329901586


При балансировке методом upsample получили

best_f1: 0.6312433581296492<br>
roc_auc: 0.8527619329901586

In [18]:
def downsample(features, target, fraction):
    features_zeros = features[target==0]
    features_ones = features[target==1]
    target_zeros = target[target==0]
    target_ones = target[target==1]
    
    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
 
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
 
    return features_downsampled, target_downsampled

In [19]:
best_f1_score = 0
best_trees = 0
best_depth = 0
features_downsample, target_downsample = downsample(features_train, target_train, 0.25)
for trees in range(1,100):
    for depth in range(1,10):
        model = RandomForestClassifier(random_state=12345, 
                                        n_estimators=trees, 
                                        max_depth=depth)
        model.fit(features_downsample, target_downsample)
    
        predictions = model.predict(features_valid)
        probabilities_one_valid = model.predict_proba(features_valid)[:, 1]
        
        if f1_score(target_valid, predictions) > best_f1_score:
            best_f1_score = f1_score(target_valid, predictions)
            roc_auc = roc_auc_score(target_valid, probabilities_one_valid)
            best_trees = trees
            best_depth = depth
        
print('depth', best_depth, 'trees', best_trees)
print('best_f1:', best_f1_score)
print('roc_auc:', roc_auc)

depth 4 trees 10
best_f1: 0.6092184368737475
roc_auc: 0.8354975834598564


При балансировке методом downsample получили 

best_f1: 0.6092184368737475 <br>
roc_auc: 0.8354975834598564

## Тестирование модели

Из прошлых пунктов мы получили, что лучшая модель - это случайный лес (depth=9, trees=88) при добалансировки с помощью аргумента class_weight. Проверим данную модель на тестовой выборке.

In [20]:
model = RandomForestClassifier(random_state=12345, 
                                n_estimators=88, 
                                max_depth=9, 
                                class_weight='balanced')
model.fit(features_train, target_train)
    
predictions = model.predict(features_test)
probabilities_one_valid = model.predict_proba(features_test)[:, 1]
        
print('best_f1:', f1_score(target_test, predictions))
print('roc_auc:', roc_auc_score(target_test, probabilities_one_valid))

best_f1: 0.6126126126126127
roc_auc: 0.8559583612538995


Получили значения:

best_f1: 0.6126126126126127<br>
roc_auc: 0.8559583612538995

Мы получили значение best_f1 свыше 0,59, что удовлетворяет нашему заданию.

## Итоговый вывод

В ходе проекта была построена и обучена модель бинарной классификации, которая поможет предсказывать «Бета-Банку» уйдет ли клиент в ближайшее время из банка или нет.<br>

В первую очередь были предобработаны данные(заполены пропуски), а затем данные были подготовлены к обучению: 
- были удалены лишние признаки, 
- категориальные рпизнаки преобразованы в количественные методом  OHE,
- данные разделены на целевой и другие признаки, 
- данные разделены на обучающую, тестовую и валидациаонную выборки в соотношении 3:1:1,
- масштабированы путем стандартизации данных

Далее рассматривались три алгоритма: Дерево решений, Случайный лес, Логистическая регрессия<br>
Каждый рассматривался для различного набора гипперпараметров, наилучшим алгоритмом в нашей задаче оказался: Случайный лес<br>

Классы в нашей задаче были несбалансированы (0:1~4:1), поэтому в работе были рассмотрены три вида добалансировки: upsampling, downsampling и с помощью аргумента class_weight.<br>

По показателю f1_score лучшей оказалась добалансировка с помощью аргумента class_weight.<br>
При этом лучшая модель была случайный лес (depth=9, trees=88)<br>

Было проведено тестирование данной модели, получены следующие результаты:
- best_f1: 0.6126126126126127
- roc_auc: 0.8559583612538995